# Import libraries 

In [0]:
import time
t = time.time()
import json
import string
import random
import math
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
import nltk
# nltk.download()
import nltk, re, time
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

In [0]:
# !pip install --upgrade gensim
from gensim.models import Word2Vec

# Load and Inspect Data

**Authentication via google auth**

In [0]:
# Code to read csv file into colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

**Upload a file from google cloud where file ids are stored as a json**

In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
file_id_data= json.loads(uploaded['file_id.json'])

User uploaded file "file_id.json" with length 356 bytes


**Download files to the kernel**

In [0]:
#2. Get the files
downloaded1 = drive.CreateFile({'id':file_id_data["train_file_id"]}) 
downloaded1.GetContentFile('train.csv')  
downloaded2 = drive.CreateFile({'id':file_id_data["test_file_id"]}) 
downloaded2.GetContentFile('test.csv') 

**Load train and test data to notebook**

In [0]:
#3. Read file as panda dataframe
train_data = pd.read_csv('train.csv') 
test_data = pd.read_csv('test.csv')

**Inspect train and test data**

In [8]:
train_data.head(20)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explan edit made usernam hardcor metallica fan...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww match background colour seem stuck thank t...,0,0,0,0,0,0
2,000113f07ec002fd,hey man realli tri edit war guy constant remov...,0,0,0,0,0,0
3,0001b41b1c6bb37e,make real suggest improv wonder section statis...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chanc rememb page,0,0,0,0,0,0
5,00025465d4725e87,congratul well use tool well talk,0,0,0,0,0,0
6,0002bcb3da6cb337,cocksuck piss around work,1,1,1,0,1,0
7,00031b1e95af7921,vandal matt shirvington articl revert pleas ban,0,0,0,0,0,0
8,00037261f536c51d,sorri word nonsens offens anyway intend write ...,0,0,0,0,0,0
9,00040093b2687caa,align subject contrari dulithgow,0,0,0,0,0,0


In [9]:
print("Shape of train data:", train_data.shape)

Shape of train data: (159571, 8)


In [10]:
test_data.head(20)

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule succes ever what hate sad mof...
1,0000247867823ef7,rfc titl fine imo
2,00013b17ad220c46,sourc zaw ashton lapland
3,00017563c3f7919a,look back sourc inform updat correct form gues...
4,00017695ad8997eb,anonym edit articl
5,0001ea8717f6de06,thank understand think high would revert witho...
6,00024115d4cbde0f,pleas add nonsens wikipedia edit consid vandal...
7,000247e83dcc1211,dear god site horribl
8,00025358d4737918,fool believ number correct number lie ponder...
9,00026d1092fe71cc,doubl redirect fix doubl redirect blank outer ...


In [11]:
print("Shape of test data:", test_data.shape)

Shape of test data: (153164, 2)


# Preprocess the text data

In [0]:
# load list of stopwords
sw = set(stopwords.words("english"))
# load teh snowball stemmer
stemmer = SnowballStemmer("english")
# translator object to replace punctuations with space
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))

In [13]:
print(sw)

{'needn', 'won', 't', 'haven', 'm', 'you', 'doing', "mightn't", 'does', 'about', 'of', "weren't", 'he', 'isn', 'with', 'yourselves', 'she', 'under', 'where', 'but', 'll', 'down', 'there', 'here', 'ma', "you'd", 'me', 'into', 'this', 'that', 're', "should've", 'nor', 'ours', 'few', 'itself', 've', "you've", 'most', 'just', 'on', 'which', 'or', 'o', 'before', 'had', 'when', "wouldn't", "it's", 'very', 'a', "isn't", "you're", "shouldn't", 'them', 'only', 'against', 'as', 'the', "mustn't", 'should', 'my', 'while', 'shouldn', 'because', 'not', 'am', 'than', "that'll", 'an', 'weren', 'during', 'will', "aren't", 'aren', 'y', 'do', 'has', 'if', 'mightn', 'him', "she's", "couldn't", 'been', 'theirs', 'wouldn', 'shan', 'both', 'and', 'himself', 'being', 'same', 'couldn', 'in', 'it', 'didn', "didn't", 'ain', 'd', 'then', 'have', 'no', 'each', 'more', "you'll", 'herself', 'yours', 'until', 'so', 'doesn', 'hasn', 'between', 'are', 'again', 'hadn', "won't", 'hers', 'whom', 'was', 'is', 'once', 'they

**Function for preprocessing text**

In [0]:
def clean_text(text):
    
    '''a function for preprocessing'''
    
    text = str(text)
    
    # replacing the punctuations with no space,which in effect deletes the punctuation marks 
    text = text.translate(translator)
    
    # remove stop word
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    text = " ".join(text)
    
    # stemming
    text = [stemmer.stem(word) for word in text.split()]
    text = " ".join(text) 
    
    # Clean the text
    text = re.sub(r"<br />", " ", text)
    text = re.sub(r"[^a-z]", " ", text)
    text = re.sub(r"   ", " ", text) # Remove any extra spaces
    text = re.sub(r"  ", " ", text)
    
    return(text)

**Clean train and test data**

In [15]:
t1 = time.time()
train_data['comment_text'] = train_data['comment_text'].apply(clean_text)
print("Finished cleaning the train set.", "Time needed:", time.time()-t1)

Finished cleaning the train set. Time needed: 86.80020904541016


In [16]:
t2 = time.time()
test_data['comment_text'] = test_data['comment_text'].apply(clean_text)
print("Finished cleaning the test set.", "Time needed:", time.time()-t2)

Finished cleaning the test set. Time needed: 76.47211194038391


**Inspect the cleaned train and test data**

In [17]:
train_data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explan edit made usernam hardcor metallica fan...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww match background colour seem stuck thank t...,0,0,0,0,0,0
2,000113f07ec002fd,hey man realli tri edit war guy constant remov...,0,0,0,0,0,0
3,0001b41b1c6bb37e,make real suggest improv wonder section statis...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chanc rememb page,0,0,0,0,0,0
5,00025465d4725e87,congratul well use tool well talk,0,0,0,0,0,0
6,0002bcb3da6cb337,cocksuck piss around work,1,1,1,0,1,0
7,00031b1e95af7921,vandal matt shirvington articl revert plea ban,0,0,0,0,0,0
8,00037261f536c51d,sorri word nonsen offen anyway intend write an...,0,0,0,0,0,0
9,00040093b2687caa,align subject contrari dulithgow,0,0,0,0,0,0


In [18]:
test_data.head(10)

,id,comment_text
0,00001cee341fdb12,yo bitch ja rule succ ever hate sad mofucka bi...
1,0000247867823ef7,rfc titl fine imo
2,00013b17ad220c46,sourc zaw ashton lapland
3,00017563c3f7919a,look back sourc inform updat correct form gues...
4,00017695ad8997eb,anonym edit articl
5,0001ea8717f6de06,thank understand think high would revert witho...
6,00024115d4cbde0f,plea add nonsen wikipedia edit consid vandal q...
7,000247e83dcc1211,dear god site horribl
8,00025358d4737918,fool believ number correct number lie ponder n...
9,00026d1092fe71cc,doubl redirect fix doubl redirect blank outer ...


# Create columns for length of the data

**A function for finding the length of text**

In [0]:
def find_length(text):
  
  text = str(text)
  
  return len(text.split())

**Create the column of text length in train and test data**

In [20]:
train_data['length'] = train_data['comment_text'].apply(find_length)
train_data.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,length
0,0000997932d777bf,explan edit made usernam hardcor metallica fan...,0,0,0,0,0,0,23
1,000103f0d9cfb60f,aww match background colour seem stuck thank t...,0,0,0,0,0,0,10
2,000113f07ec002fd,hey man realli tri edit war guy constant remov...,0,0,0,0,0,0,21
3,0001b41b1c6bb37e,make real suggest improv wonder section statis...,0,0,0,0,0,0,52
4,0001d958c54c6e35,sir hero chanc rememb page,0,0,0,0,0,0,5
5,00025465d4725e87,congratul well use tool well talk,0,0,0,0,0,0,6
6,0002bcb3da6cb337,cocksuck piss around work,1,1,1,0,1,0,4
7,00031b1e95af7921,vandal matt shirvington articl revert plea ban,0,0,0,0,0,0,7
8,00037261f536c51d,sorri word nonsen offen anyway intend write an...,0,0,0,0,0,0,38
9,00040093b2687caa,align subject contrari dulithgow,0,0,0,0,0,0,4


In [21]:
test_data['length'] = test_data['comment_text'].apply(find_length)
test_data.head(10)

,id,comment_text,length
0,00001cee341fdb12,yo bitch ja rule succ ever hate sad mofucka bi...,43
1,0000247867823ef7,rfc titl fine imo,4
2,00013b17ad220c46,sourc zaw ashton lapland,4
3,00017563c3f7919a,look back sourc inform updat correct form gues...,15
4,00017695ad8997eb,anonym edit articl,3
5,0001ea8717f6de06,thank understand think high would revert witho...,8
6,00024115d4cbde0f,plea add nonsen wikipedia edit consid vandal q...,17
7,000247e83dcc1211,dear god site horribl,4
8,00025358d4737918,fool believ number correct number lie ponder n...,44
9,00026d1092fe71cc,doubl redirect fix doubl redirect blank outer ...,21


# Save the modified train and test data

**Save the cleaned train data in google drive directory**

In [0]:
train_data.to_csv("./modified_train_data.csv", header=True, index=False)
uploaded = drive.CreateFile({'title': 'modified_train_data.csv', 'parents':[{u'id': file_id_data["download_folder1_id"]}]})
uploaded.SetContentFile("modified_train_data.csv")
uploaded.Upload()

**Save the cleaned test data in google drive directory**

In [0]:
test_data.to_csv("./modified_test_data.csv", header=True, index=False)
uploaded = drive.CreateFile({'title': 'modified_test_data.csv', 'parents':[{u'id': file_id_data["download_folder1_id"]}]})
uploaded.SetContentFile("modified_test_data.csv")
uploaded.Upload()